In [3]:
# EDA
import pandas as pd
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# ML
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Otimizacao HP
import optuna

# Carga de dados

In [4]:
# Carregar dataset
df_clientes = pd.read_csv('datasets/dataset_segmento_clientes.csv')

In [6]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [8]:
# remover coluna segmento_de_cliente
df_clientes = df_clientes.drop(columns=['segmento_de_cliente'])

In [10]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [9]:
df_clientes.head()

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8


# EDA

In [13]:
# Distribuicao da variável inovacao
percentual_inovacao = df_clientes['inovacao'].value_counts(normalize=True) * 100
px.bar(percentual_inovacao, title='Distribuição da variável inovação', labels={'value': 'Percentual (%)', 'index': 'Inovação'}, color=percentual_inovacao.index)

In [14]:
# Teste ANOVA (Análise de Variância)
# Verifica se há diferença significativa entre as médias de inovação para cada segmento de cliente
# Suposições / Pressupostos:
# - Observacoes independentes
# - Variável depedendente seja contínua
# - Segue uma distribuição normal
# - Homogeneidade das variâncias
# - Amostras sejam de tamanhos iguais

In [21]:
# Chegar se as variâncias (faturamento) entre os grupos (inovacao) são homogêneas
# Aplicar teste de Bartlett
# H0: As variâncias são homogêneas
# H1: As variâncias não são homogêneas
from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna inovacao
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()]

# Executar teste de Bartlett
bartlett_test_statistic, barlett_p_value = bartlett(*dados_agrupados)

# Exibir os resultados do teste
print(f'Estatística de teste de Bartlett: {bartlett_test_statistic}')
print(f'p-valor do teste de barlett: {barlett_p_value}')

Estatística de teste de Bartlett: 10.901203117231173
p-valor do teste de barlett: 0.2825418295490583


Como o p-valor é maior que 0.05 não podemos rejeitar H0, ou seja, as variâncias são iguais.

In [23]:
# Executar o teste de Shapiro-Wilk para verificar a normalidade dos dados
# H0: Os dados seguem uma distribuição normal
# H1: Os dados não seguem uma distribuição normal
from scipy.stats import shapiro

# Executar o teste de Shapiro-Wilk
shapiro_test_statistic, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

# Exibir os resultados do teste
print(f'Estatística de teste de Shapiro-Wilk: {shapiro_test_statistic}')
print(f'p-valor do teste de Shapiro-Wilk: {shapiro_p_value}')

Estatística de teste de Shapiro-Wilk: 0.9959857602472714
p-valor do teste de Shapiro-Wilk: 0.2351345103439314


Como o p-valor é maior que 0.05 não podemos rejeitar H0, ou seja, os dados seguem uma distribuição normal.

In [24]:
# Aplicar teste ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0: Não há diferenças significativas entre as médias dos grupos
# H1: Há pelo menos uma diferença significativa entre as médias dos grupos

from pingouin import welch_anova

aov = welch_anova(data=df_clientes, dv='faturamento_mensal', between='inovacao')

# Exibir os resultados do teste - ANOVA
print(f'Estatística de teste ANOVA: {aov["F"][0]}')
print(f'p-valor do teste ANOVA: {aov["p-unc"][0]}')


Estatística de teste ANOVA: 1.126983619406169
p-valor do teste ANOVA: 0.34526211273912644


Como p-valor é maior que 0.05 não podemos rejeitar H0, ou seja, as médias são iguais. Ou seja, independente do nível de inovação que a empresa esteja posicionada quando agrupa as empresas nesses níveis não há diferença significativa entre as médias de faturamento entre esses níveis.

# Treinar o algoritmo KMeans

In [27]:
df_clientes.head()

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8


In [30]:
# Selecionar as Colunas para clusterização
X = df_clientes.copy()

# Separando variáveis numéricas, categoricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# Aplicar transformações
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

# Transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [31]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])